In [35]:
import pandas as pd
import numpy as np
import glob, sys, os, time , itertools , warnings
from bs4 import BeautifulSoup
from tqdm import tqdm
import multiprocessing as mp
# from DataProcessing import 

warnings.filterwarnings('ignore')

In [57]:
path_to_unemp = path_to_database+'/US-Unemployement/us-unemployement20142018.xlsx'

unempdf = pd.concat([pd.read_excel(path_to_unemp,sheet_name=i,index_col=0) for i in range(4)],axis=1)

col_to_filt = [col for col in unempdf.columns if 'rate' in col]

unempdf= unempdf[col_to_filt]
unempdf.columns = ['unemp-'+col for col in unempdf.columns]
unempdf.index = [ind.replace('Metropolitan Statistical Area','').strip() for ind in unempdf.index]
unempdf.sort_index()

,unemp-rate-2014,unemp-rate-2015,unemp-rate-2016,unemp-rate-2017
"Abilene, TX",4.3,3.8,3.9,3.7
"Akron, OH",5.9,5.0,5.1,5.0
"Albany, GA",8.3,7.0,6.2,5.4
"Albany, OR",8.1,6.7,5.7,4.7
"Albany-Schenectady-Troy, NY",5.1,4.5,4.1,4.3
...,...,...,...,...
"Yakima, WA",8.7,8.1,7.5,6.8
"York-Hanover, PA",5.4,4.8,4.6,4.3
"Youngstown-Warren-Boardman, OH-PA",6.7,6.1,6.4,6.7
"Yuba City, CA",12.1,10.1,9.3,8.2


In [58]:
path_to_income = path_to_database+'/US-Personal-Income/lapi1118msa_0.xlsx'
income = pd.read_excel(path_to_income,sheet_name=1,index_col=0)
income.sort_index()

,income-2015,income-2016,income-2017
Metropolitan Statistical Areas,,,
"Abilene, TX",40886,40558,41326
"Akron, OH",45108,45704,47511
"Albany, GA",34770,35629,36805
"Albany, OR",37955,39061,40380
"Albany-Schenectady-Troy, NY",52649,53638,55848
...,...,...,...
"Yakima, WA",39233,39968,41331
"York-Hanover, PA",44687,45790,47427
"Youngstown-Warren-Boardman, OH-PA",38976,39513,40649


In [59]:
path_to_pop = "/home/vlad/csv/US-Population/cbsa-est2018-alldata.xlsx"
pop = pd.read_excel(path_to_pop,sheet_name=1#,index_col=0
                     )
pop['NAME'] =pop['NAME'].str.strip()
pop = pop.set_index('NAME')
pop.sort_index()

,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018
NAME,,,,,,,,,,,
"Abbeville County, SC",25417,25399,25310,25060,25000,24880,24775,24775,24633,24560,24541
"Aberdeen, SD",40602,40603,40717,40962,41576,42155,42220,42404,42849,43235,43191
"Aberdeen, WA",72797,72798,72843,72357,71770,71046,70783,71027,71572,72553,73901
"Abilene, TX",165252,165246,165583,166616,167447,167472,168355,169704,170018,170516,171451
"Acadia Parish, LA",61773,61787,61875,61853,61991,62294,62664,62681,62793,62514,62190
...,...,...,...,...,...,...,...,...,...,...,...
"Yuma County, AZ",195751,195750,197138,202883,202589,202589,204283,205456,207520,209756,212128
"Yuma, AZ",195751,195750,197138,202883,202589,202589,204283,205456,207520,209756,212128
"Zanesville, OH",86074,86086,86213,86236,85786,85640,85900,86174,85973,86148,86183


In [74]:
unempdf.shape

(390, 4)

In [77]:
joint_full = pd.concat([unempdf,income],axis=1).join(pop)

In [79]:
joint_full[['unemp-rate-2015', 'unemp-rate-2016',
       'unemp-rate-2017', 'income-2015', 'income-2016', 'income-2017',
        'POPESTIMATE2015', 'POPESTIMATE2016',
       'POPESTIMATE2017', 'POPESTIMATE2018']]

,unemp-rate-2015,unemp-rate-2016,unemp-rate-2017,income-2015,income-2016,income-2017,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018
"Abilene, TX",3.8,3.9,3.7,40886.0,40558.0,41326.0,169704.0,170018.0,170516.0,171451.0
"Akron, OH",5.0,5.1,5.0,45108.0,45704.0,47511.0,704448.0,703690.0,704367.0,704845.0
"Albany, GA",7.0,6.2,5.4,34770.0,35629.0,36805.0,153604.0,152353.0,151293.0,153009.0
"Albany, OR",6.7,5.7,4.7,37955.0,39061.0,40380.0,120244.0,122730.0,124977.0,127335.0
"Albany-Schenectady-Troy, NY",4.5,4.1,4.3,52649.0,53638.0,55848.0,879131.0,879790.0,882130.0,883169.0
...,...,...,...,...,...,...,...,...,...,...
"Yakima, WA",8.1,7.5,6.8,39233.0,39968.0,41331.0,248065.0,249691.0,250377.0,251446.0
"York-Hanover, PA",4.8,4.6,4.3,44687.0,45790.0,47427.0,441800.0,443777.0,445722.0,448273.0
"Youngstown-Warren-Boardman, OH-PA",6.1,6.4,6.7,38976.0,39513.0,40649.0,548873.0,544688.0,541875.0,538952.0
"Yuba City, CA",10.1,9.3,8.2,39719.0,40567.0,41673.0,169471.0,170900.0,173213.0,174848.0


In [81]:
joint_full.index.tolist()

['Abilene, TX',
 'Akron, OH',
 'Albany, GA',
 'Albany, OR',
 'Albany-Schenectady-Troy, NY',
 'Albuquerque, NM',
 'Alexandria, LA',
 'Allentown-Bethlehem-Easton, PA-NJ',
 'Altoona, PA',
 'Amarillo, TX',
 'Ames, IA',
 'Anchorage, AK',
 'Ann Arbor, MI',
 'Anniston-Oxford-Jacksonville, AL',
 'Appleton, WI',
 'Asheville, NC',
 'Athens-Clarke County, GA',
 'Atlanta-Sandy Springs-Roswell, GA',
 'Atlantic City-Hammonton, NJ',
 'Auburn-Opelika, AL',
 'Augusta-Richmond County, GA-SC',
 'Austin-Round Rock, TX',
 'Bakersfield, CA',
 'Baltimore-Columbia-Towson, MD',
 'Bangor, ME',
 'Bangor, ME Metropolitan NECTA',
 'Barnstable Town, MA',
 'Barnstable Town, MA Metropolitan NECTA',
 'Baton Rouge, LA',
 'Battle Creek, MI',
 'Bay City, MI',
 'Beaumont-Port Arthur, TX',
 'Beckley, WV',
 'Bellingham, WA',
 'Bend-Redmond, OR',
 'Billings, MT',
 'Binghamton, NY',
 'Birmingham-Hoover, AL',
 'Bismarck, ND',
 'Blacksburg-Christiansburg-Radford, VA',
 'Bloomington, IL',
 'Bloomington, IN',
 'Bloomsburg-Berwick